# Imagenet Processing in parallel

In [5]:
%matplotlib inline
from importlib import reload
import utils2; reload(utils2)
from utils2 import *
import utils2_ted; reload(utils2_ted)
from utils2_ted import *

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
from bcolz_array_iterator import BcolzArrayIterator
from tqdm import tqdm

In [7]:
limit_mem()

This is where our full dataset lives. It's slow spinning discs, but lots of room!

**NB:** We can easily switch to and from using a sample. We'll use a sample for *everything*, except the final complete processing (which we'll use fast/expensive compute for, and time on the sample so we know how long it will take).

In [8]:
path = '../../../Data/imagenet-sample-train/'
#path = '../data2/train/'

This is on a RAID 1 SSD for fast access, so good for resized images and feature arrays

In [9]:
dpath = '../../../Data/'
#dpath = '../data2/'

Note that either way, AWS isn't going to be a great place for doing this kind of analysis - putting a model into production will cost at minimum $600/month for a P2 instance. For that price you can **buy** a GTX 1080 card, which has **double** the performance of the AWS P2 card! And you can set up your slow full data RAID 5 array and your fast preprocessed data RAID 1 array just as you like it. Since you'll want your own servers for production, you may as well use them for training, and benefit from the greater speed, lower cost, and greater control of storage resources.

You can put your server inside a colo facility for very little money, paying just for the network and power. (Cloud providers aren't even allowed to provide GTX 1080's!)

There's little need for distributed computing systems for the vast majority of training and production needs in deep learning.

## Get word vectors

First we need to grab some word vectors, to use as our dependent variable for the image model (so that the image vectors and word vectors will be in the same space). After loading the word vectors, we'll make sure that the names of the wordnet/imagenet are in the word list.

* Be careful not to just follow paper's approach - e.g. here word2vec better than custom wikipedia vectors. word2vec has multi-word tokens like 'golden retriever'
* Take evaluations shown in papers with a grain of salt, and do your own tests on important bits. E.g. DeVISE (because it's an older paper) used an old and inaccurate image model, and poor word vectors, so recent papers that compare to it aren't so relevent

In [6]:
from gensim.models import word2vec

w2v_path= dpath+'GoogleNews-vectors-negative300'

In [19]:
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = word2vec.KeyedVectors.load_word2vec_format(w2v_path+".bin", binary=True)
model.save_word2vec_format(w2v_path+'.txt', binary=False)

In [7]:
lines = open(w2v_path+'.txt', encoding='utf-8').readlines()

In [8]:
def parse_web2v(l):
    i = l.index(' ')
    return l[:i], np.fromstring(l[i:], dtype=np.float32, sep=' ')

In [9]:
w2v_list = list(map(parse_web2v, lines[1:]))

In [12]:
len(w2v_list)

3000000

In [1]:
from sklearn.externals import joblib

In [ ]:
#joblib.dump(w2v_list, path+'w2v1.pkl')

In [10]:
w2v_list = joblib.load(path+'w2v1.pkl')

ValueError: EOF: reading array data, expected 1200 bytes got 0

In [11]:
pickle.dump(w2v_list, open(path+'w2v1.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

MemoryError: 

In [42]:
save_big_data(path, 'w2v1', w2v_list)

MemoryError: 

In [35]:
w2v_list = load_big_data(path, 'w2v1')

KeyError: 'w2v1.klp'

In [25]:
lst = list(np.random.rand(10000))

In [26]:
len(lst)

10000

In [27]:
save_big_data(path, 'lst', lst)

In [29]:
del lst

NameError: name 'lst' is not defined

In [30]:
lst

NameError: name 'lst' is not defined

In [31]:
lst = load_big_data(path, 'lst')

In [33]:
len(lst)

10000